In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Carregar a arquitetura pré-treinada VGG16, excluindo as camadas densas (topo) e mantendo os pesos do Imagenet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Congelar os pesos das camadas convolucionais da VGG16
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas densas no topo da VGG16 para classificação binária (gato ou cachorro)
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Construir o modelo final
model = Model(inputs=base_model.input, outputs=output)

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Diretórios de treinamento e validação
train_dir = '/tmp/cats-v-dogs/training'
validation_dir = '/tmp/cats-v-dogs/testing'

# Configurar os geradores de dados para pré-processamento e aumento de dados
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

# Treinar o modelo
history = model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=50)